# How to Operate

## Press 'a' to activate and 'd' to deactive
## Close ring and pinky fingers to start mouse controller
###    -> join Index and Middle fingers to move mouse  (ring and pinky fingers closed)
###    -> join Index and Thumb fingers to Left Click   (ring and pinky fingers closed)
###    -> join Middle and Thumb fingers to Right Click (ring and pinky fingers closed)
###    -> join Middle, Index and Thumb fingers to Scroll Up/Down (ring and pinky fingers closed)

In [ ]:
# pip install pynput
# pip install pyautogui

## Main Code

In [1]:
import cv2
import mediapipe as mp
import math
import numpy
import pyautogui
from collections import deque

# Get screen size
screenWidth, screenHeight = pyautogui.size()

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# A deque to store previous cursor positions for smoothing
smooth_factor = 5  # Adjust this factor for more smoothing (higher = smoother)
prev_positions = deque(maxlen=smooth_factor)  # A buffer of recent positions

# Activation flag
system_active = False

# Function to get a smoothed cursor position
def get_smoothed_position(newX, newY):
    # Add new position to the buffer
    prev_positions.append((newX, newY))
    
    # Calculate the average position from the buffer
    avgX = sum([pos[0] for pos in prev_positions]) / len(prev_positions)
    avgY = sum([pos[1] for pos in prev_positions]) / len(prev_positions)
    
    return avgX, avgY

# Function to check if a finger is open (Tip lower than base)
def is_finger_open(lmList, tip_id, pip_id):
    return lmList[tip_id][2] < lmList[pip_id][2]  # Compare y-coordinates (Tip should be lower than knuckle)

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert the image to RGB
    results = hands.process(imgRGB)  # Process the RGB image

    # Listen for key presses to activate/deactivate the system
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):  # Press 'a' to activate the system
        system_active = True
        print("System Activated")
    elif key == ord('d'):  # Press 'd' to deactivate the system
        system_active = False
        print("System Deactivated")

    if system_active and results.multi_hand_landmarks:  # If hand landmarks are detected and system is active
        for handLms in results.multi_hand_landmarks:  # Loop through each detected hand
            lmList = []
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)  # Convert landmark coordinates to pixel values
                lmList.append([id, cx, cy])

                # Draw landmarks on the image
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

            if lmList:
                # Check if index and middle fingers are open
                index_open = is_finger_open(lmList, 8, 6)  # Tip of index finger (8), base of index finger (6)
                middle_open = is_finger_open(lmList, 12, 10)  # Tip of middle finger (12), base of middle finger (10)
                middle_open = True
                index_open = True
                # Check if other fingers are closed (ring, pinky, and thumb)
                ring_closed = not is_finger_open(lmList, 16, 14)  # Ring finger
                pinky_closed = not is_finger_open(lmList, 20, 18)  # Pinky finger

                # Check for left click (thumb touching index finger)
                thumb_x, thumb_y = lmList[4][1], lmList[4][2]
                index_x, index_y = lmList[8][1], lmList[8][2]
                middle_x, middle_y = lmList[12][1], lmList[12][2]

                dist_thumb_index = math.hypot(thumb_x - index_x, thumb_y - index_y)  # Distance between thumb and index finger
                dist_thumb_middle = math.hypot(thumb_x - middle_x, thumb_y - middle_y)  # Distance between thumb and middle finger

                # Only perform action if index and middle fingers are open, and others are closed
                if index_open and middle_open and ring_closed and pinky_closed:
                    x1, y1 = lmList[8][1], lmList[8][2]  # Tip of the index finger
                    x2, y2 = lmList[12][1], lmList[12][2]  # Tip of the middle finger
                    cv2.circle(img, (x1, y1), 15, (255, 0, 0), cv2.FILLED)
                    cv2.circle(img, (x2, y2), 15, (255, 0, 0), cv2.FILLED)
                    cv2.circle(img, (thumb_x, thumb_y), 15, (255, 255, 255), cv2.FILLED)

                    z1, z2 = (x1 + x2) // 2, (y1 + y2) // 2  # Midpoint between index and middle fingers
                    length = math.hypot(x2 - x1, y2 - y1)

                    # Scale webcam coordinates to screen size
                    screenX = numpy.interp(x1, [0, w], [0, screenWidth])
                    screenY = numpy.interp(y1, [0, h], [0, screenHeight])
                    

                    if length < 35:
                        # Get smoothed cursor position
                        smoothedX, smoothedY = get_smoothed_position((1870 - screenX) * 1.5, screenY * 1.5)

                        # Move the cursor to the smoothed position
                        pyautogui.moveTo(smoothedX, smoothedY)
                        cv2.circle(img, (z1, z2), 15, (255, 255, 255), cv2.FILLED)


                    # Scroll condition (thumb + index + middle fingers close)
                    if dist_thumb_index < 40 and dist_thumb_middle < 40:
                        cv2.putText(img, "Scrolling", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                        # Track the hand's vertical movement (Y-coordinate)
                        if prev_scroll_y is not None:
                            if screenY < prev_scroll_y - 10:  # Moving hand up
                                pyautogui.scroll(20)  # Scroll up
                                cv2.putText(img, "Scrolling Up", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                            elif screenY > prev_scroll_y + 10:  # Moving hand down
                                pyautogui.scroll(-20)  # Scroll down
                                cv2.putText(img, "Scrolling Down", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                        # Update previous scroll Y-position
                        prev_scroll_y = screenY

                    # Left click (thumb + index finger close)
                    elif dist_thumb_index < 40:
                        cv2.circle(img, (thumb_x, thumb_y), 15, (255, 255, 255), cv2.FILLED)
                        cv2.circle(img, (index_x, index_y), 15, (255, 255, 255), cv2.FILLED)
                        pyautogui.click()  # Perform left click
                        cv2.putText(img, "Left Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                        # prev_scroll_y = None  # Reset when the fingers are not in the scroll position
                    
                    # Right click (thumb + middle finger close)
                    elif dist_thumb_middle < 40:
                        cv2.circle(img, (thumb_x, thumb_y), 15, (255, 255, 255), cv2.FILLED)
                        cv2.circle(img, (middle_x, middle_y), 15, (255, 255, 255), cv2.FILLED)
                        pyautogui.rightClick()  # Perform right click
                        cv2.putText(img, "Right Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                        # prev_scroll_y = None  # Reset when the fingers are not in the scroll position
                        
                    else:
                        prev_scroll_y = None  # Reset when the fingers are not in the scroll position

    # Display the image
    cv2.imshow("Image", img)

    # Break the loop when 'q' is pressed
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\debab\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


System Activated
System Deactivated
System Activated
System Activated
